# 1. Instalar e Importar Librerías Necesarias

En esta sección instalaremos e importaremos todas las librerías necesarias para el procesamiento de PDFs, vectorización, tokenización y uso de modelos ligeros multilenguaje compatibles con Ollama.

In [ ]:
# Instalar librerías necesarias
%pip install PyPDF2 sentence-transformers langchain faiss-cpu pdf2image pytesseract pillow opencv-python

# Importar librerías
import os
import subprocess
from pdf2image import convert_from_path
import pytesseract
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import cv2
from PIL import Image
from pathlib import Path


# 2. Configuración y seteo de binarios

En esta sección construiremos todas las configuraciones necesarias, path a binario y demás

In [ ]:

BASE_DIR = Path().resolve()
PDF_FOLDER = os.path.join(BASE_DIR, "pdfs")
print(f"PDF_FOLDER: {PDF_FOLDER}")

INDEX_PATH = os.path.join(BASE_DIR, "faiss_index.bin")
CHUNKS_PATH = os.path.join(BASE_DIR, "chunks.pkl")
EMBEDDINGS_PATH = os.path.join(BASE_DIR, "embeddings.npy")
OUTPUT_FOLDER = os.path.join(BASE_DIR, "ocr_textos")


#tesseract
tesseract_cmd = os.path.join(os.getcwd(), "tesseract", "tesseract.exe")
pytesseract.pytesseract.tesseract_cmd = tesseract_cmd

#poppler
poppler_path = os.path.join(os.getcwd(), "poppler", "bin")

required_bins = ["pdfinfo.exe", "pdftoppm.exe"]
for b in required_bins:
    if not os.path.exists(os.path.join(poppler_path, b)):
        raise FileNotFoundError(f"No se encontró {b} en {poppler_path}")

# 3. Cargar modelo multilenguaje

En esta sección se carga un modelo para realizar los embeddings. Debe ser el mismo con el cual se lee después

In [ ]:
model_name = "paraphrase-multilingual-mpnet-base-v2"
embedder = SentenceTransformer(model_name)

# 4. Función para mejorar el contraste de los pdf

En esta sección se mejora el pdf, realzando el contraste para mejorar la identificación del texto

In [ ]:
def preprocess_image(pil_image):
    cv_image = np.array(pil_image)
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2GRAY)
    # Aplicar umbral para mejorar contraste
    _, thresh = cv2.threshold(cv_image, 180, 255, cv2.THRESH_BINARY)
    return Image.fromarray(thresh)

# 5. Función OCR de imágenes dentro del pdf

En esta sección se revisan si hay imagenes en pdf para extraer texto

In [ ]:
def extract_text_from_image_pdf(pdf_file, psm=6):
    pages = convert_from_path(pdf_file, dpi=300, poppler_path=poppler_path)
    all_chunks = []

    pdf_name = os.path.splitext(os.path.basename(pdf_file))[0]
    txt_output = os.path.join(OUTPUT_FOLDER, f"{pdf_name}ocr.txt")

    with open(txt_output, "w", encoding="utf-8") as f:
        f.write(f"========== OCR de {os.path.basename(pdf_file)} ==========\n")

        for i, page in enumerate(pages):
            processed_page = preprocess_image(page)
            text = pytesseract.image_to_string(processed_page, lang='spa', config=f'--psm {psm}').strip()

            f.write(f"\n--- Página {i + 1} ---\n")
            if text:
                f.write(text + "\n")
                words = text.split()
                page_chunks = [' '.join(words[j:j + 200]) for j in range(0, len(words), 200)]
                all_chunks.extend(page_chunks)
                print(f"📄 {os.path.basename(pdf_file)} - Página {i + 1}: {len(words)} palabras extraídas")
            else:
                f.write("[Sin texto detectado]\n")
                print(f"⚠️ {os.path.basename(pdf_file)} - Página {i + 1}: sin texto detectado")

    print(f"📝 Texto OCR guardado en: {txt_output}")
    return all_chunks


# 6. Procesamos todos los pdfs

En esta sección se recolentan los pdfs y se procesan

In [ ]:
pdf_files = [os.path.join(PDF_FOLDER, f) for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]

all_chunks = []
for pdf_file in pdf_files:
    chunks = extract_text_from_image_pdf(pdf_file)
    print(f"{os.path.basename(pdf_file)} -> {len(chunks)} chunks generados\n")
    all_chunks.extend(chunks)

print(f"📦 Total de chunks generados: {len(all_chunks)}")
print(f"📁 Archivos OCR guardados en: {OUTPUT_FOLDER}")

# 7. Generación de embeddings

En esta sección se generan los embeddings

In [ ]:
if len(all_chunks) == 0:
    raise ValueError("No se generaron chunks. Revisa los PDFs y el OCR.")

embeddings = embedder.encode(all_chunks, show_progress_bar=True, convert_to_numpy=True)
print(f"Se generaron {embeddings.shape[0]} vectores.")


# 8. Creamos el indice FAISS

En esta sección se crea el indice FAISS (basicamente es para manipular mejor los vectores, obtenerlos eficientemente bla bla)

In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f"Índice FAISS creado con {index.ntotal} vectores.")

# 9. Guardamos indices y datos

En esta sección se guardan los indices

In [ ]:
faiss.write_index(index, INDEX_PATH)
np.save(EMBEDDINGS_PATH, embeddings)
with open(CHUNKS_PATH, "wb") as f:
    pickle.dump(all_chunks, f)
print("FAISS index, embeddings y chunks guardados correctamente.")

# 10. Revisamos cuantos chunks se crearon

En esta sección revisamos cuantos chunks se crearon

In [ ]:
with open(CHUNKS_PATH, "rb") as f:
    loaded_chunks = pickle.load(f)
print(f"Número de chunks cargados: {len(loaded_chunks)}")
print("Ejemplo de chunk:", loaded_chunks[0][:200], "...")